In [4]:
EXTRACT_USER_PROMPT = """
We have millions of profiles in our database, dated at today (assuming its 2025). We use the following guidelines to create a query to search for them:

**skill**: Which broad skills, expertise, or specializations mentioned in the prompt that are required for the role (e.g., [Python, project management, recruiting, machine learning, sales]). Include both technical and soft skills. Ensure that terms related to regulatory expertise or compliance are categorized as skills, even if they relate to specific industries or sectors. Functional areas of expertise can be considered skills if they describe specialization (e.g., Accounting, Marketing). The skill should not have any filler words; remove adjectives or descriptive phrases before the core skill. Focus on the core concept rather than specific acts (e.g., "Agile-based project management" becomes "Project management" and "Agile"), and ensure the skill is grammatically concise. Also, infer any skills or keywords from the prompt context. For executive titles, skills are often not required as the role is self-defining.

**location**: Identify and get any geographic locations or regions specified in the prompt (e.g., New York, Europe, California). For surrounding regions, include the target region being referred to. For example, for "Get people in countries close to Egypt," the region will be "Countries near Egypt," not "Egypt." Do not create separate locations if a city and state or state and country are mentioned together (e.g., "Paris, USA" remains a single location). Understand if the user wants nearby areas. (not gotten if mentioned in context of ethnicities we cater to)

**name**: Which names are required (e.g., ["John Doe", "Will Smith"]). Exclude names of companies, brands, or institutions (e.g., "Spencer Stuart"). Focus solely on human names. If this is applied, all profiles of these names will be shown only.

**education**: Get the degree and its major (e.g., {"degree": "Bachelor's", "major", "Computer Science"}). If a major is not specified, you can still mention the degree (e.g., {{"degree": "Bachelor's"}} for "graduation"). Degrees should only be from this list: ["Associate," "Bachelor's," "Master's," "Doctorate," "Diploma," "Certificate"], while majors can vary. If a type of major is mentioned (e.g., "Degrees in the creative arts"), get known majors related to that type.

**school**: Get the schools, colleges, or universities required by the user (e.g., ["Stanford University", "Yale University"]). Try your best to fulfill the user's requirements by returning a list of actual names, if the user has mentioned their requirements for the schools.

**company_tenure**: Get the length of time the candidate should have been in their CURRENT company or industry, if specified. Only consider durations referring specifically to tenure in the current company or industry and get this information if explicitly mentioned. Ignore cumulative experiences across different companies. Return a min and max value, where the default min is 0 and the default max is 60 [0, 60]. Ignore tenures in past companies.

**role_tenure**: Get the length of time the candidate should have been in their CURRENT role or position, if specified. Only consider durations referring specifically to tenure in the current role and get this information if explicitly mentioned. Ignore cumulative experiences across different roles. Return a min and max value, with default min as 0 and default max as 60 [0, 60]. Ignore tenures in past roles.

**total_working_years**: Get the required or desired overall career duration. Express this as a numeric range of years (e.g., [2, 5], where 2 is the minimum and 5 is the maximum). The default maximum is 60, with the default minimum as -1. If no minimum or maximum is mentioned, default to [0 60]. If a single value is given without implying a min or max (e.g., "Total 10 years of experience"), return a range where the min is the value minus 1 and the max is the value plus 1 (e.g., [9, 11]).

**gender**: If only females are required by the user, gender should be ['Female'] so that profiles can be filtered.

**age**: Get age ranges if the user wants any age that fits within these categories: ["Under 25", "Over 50", "Over 65"]. Extract only if explicitly mentioned or heavily implied (e.g., the query asks for "young people" or "elderlies").

**ethnicity**: Get ONLY the required ethnicities mentioned in the prompt, choosing from this list: ["Asian", "African", "Hispanic", "Middle Eastern", "South Asian", "Caucasian"] (south east asians are South Asian; blacks are Africans; mexians, latinos/mexicans are Hispanic). Extract these only if explicitly mentioned in the context of ethnicity, not location or region. Ignore other ethnicities.

**current_ownership**: Get the ownership type from the list: ["Public," "Private," "VC Funded," "Private Equity Backed"] (Private Equity Backed refers to an investee company not investor company), but only when it is explicitly mentioned in the 'current' context. Only required when explicitly referred to (not to be extracted from queries like 'get people from startups,' 'get people from fortune 500,' etc as these queries are not asking for the ownership explicitly). Apply this filter only when the user explicitly requests ownership-based filtering. When applied, the filter will display only profiles currently associated with companies of the specified ownership type, excluding all others.

**job_title**: Identify all relevant job roles for searching. When a skill is mentioned in context, evaluate whether treating it solely as a skill would produce optimal results. If not, consider applying related job roles, but only if commonly recognized positions would satisfy the search criteria. Make a logical assessment of whether inferring additional roles would maintain search accuracy without reducing recall. For example, in the query "Find executives who are CEOs with experience in digital marketing," the phrase "digital marketing" should be treated as a skill rather than generating additional executive titles, since this would maintain focus on the specified CEO role while properly categorizing the digital marketing expertise. roles or overlapping titles, so the focus remains solely on the current "CEOs" while "digital marketing" would fit better as a skill only. For each inferred job title, explain how just adding a relevant skill wouldn't be enough.

**management_level**: Get all management levels if required by the user. Our available management levels are: ["Partners", "Founder or Co-founder", "Board of Directors", "C-Suite/Chiefs", "President", "Executive VP or Sr. VP", "Manager", "Head", "Senior Partner", "Junior Partner", "VP", "Director", "Senior (All Senior-Level Individual Contributors)", "Mid (All Mid-Level Individual Contributors)", "Junior (All Junior-Level Individual Contributors)"]. Levels should only be extracted if the user is asking for the ENTIRE management domain (eg 'CEO' does not cover the complete management domain of C-Suite/Chiefs). If "Directors of engineering" is required, and "Director" is also selected as a management level then all directors would also show in results (which would be wrong in this case) even if they are not directors of engineering, so precision would be highly reduced. Avoid this.

<Note>
    1. A management level is chosen only if it encompasses the entire domain. 
        - Example 1: "Board Chair" is a subset of "Board of Directors," so it will be treated as a job title.
        - Example 2: "VP of Engineering" is a subset of "VP," so it will also be treated as a job title.
        - Example 3: "Director of Microsoft" or "Directors in the automotive industry" has requires all directors which fits the level of "Director" specificity.

    2. Key phrases in a prompt refer to titles and functions (if mentioned). If a function is specified, it is classified as a "Job Title." For example:
        - Query: "CEO and VP of Engineering at Microsoft."
            Here, "CEO" and "VP of Engineering" are key phrases. Only "VP of Engineering" is treated as a job title due to its specificity, while "VP" cannot stand alone as a key phrase.
        - Even if a function isn't specified but the title doesn't cover the entire domain of the management level then it will be a job title.
    3. Management levels should not just be inferred using other information. Also, company names or wildcards cant be used in job titles.
    4. If a user wants all “executives” without a function, it includes "CSuite," "Executive VP or Sr. VP," and "VP." If "executive" is used with a function (e.g., "Marketing Executives"), then titles like "CMO," "Chief Marketing Officer," "Senior VP of Marketing," "VP of Marketing," etc. apply. The word "executive" or "executives" is never included by itself as a title or management level.
        
    Query: "Give me VPs working in Microsoft"
    • KEY PHRASE: "VPs" (whole phrase, not split).
    • Management Level Focus: "VPs" represents the Vice President rank in the predefined set, covering the full domain of “VP.” Emphasis is on their position in the hierarchy. 
    • Job Title Focus: In the query above, VP is not a job title. However, if a clearly stated business function is mentioned (e.g., "VP of Marketing") was mentioned, then it would have been a job title. "Microsoft" is not a business function but an organization. 

    Query: "The CFOs working in google or facebook"
    • KEY PHRASE: "CFOs" (whole phrase, not split).
    • Management Level Focus: "CFOs" doesn’t cover the complete 'C-Suite/Chiefs' so it isn't a management level.
    • Job Title Focus: CFO is a single finance role, so it’s a job title.
</Note>

The lists you generate are used for setting filters and cannot process wildcards, special characters. No other filter exists (exact companies are ignored), except for the ones above. Cater to the requirements of the user intelligently, otherwise ez scene.

<special_instructions>
    For job titles, management levels and locations classify each entity as 'CURRENT', 'PAST', 'BOTH'. Then, for all (titles, levels and industries) also determine the main Event which can be 'CURRENT', 'PAST', 'CURRENT OR PAST', 'CURRENT AND PAST'. If all entities are in current then event has to be current and if all entities are in past, event has to be past. An entity should be 'BOTH' only if past and current temporal aspects clearly satisfies the query's demand (for example "Get me software engineers" has no need for past temporal aspect; "experience as a software engineer" doesn't clear an ongoing experience or past). First give your reasoning for each entity, and then the main event event. In case some entities are PAST, CURRENT or if any entity is BOTH then the event will either be "CURRENT OR PAST" or "CURRENT AND PAST". OR and AND reflects whether a sequential progression from the past entities to the current entities is a requirement; If the CURRENT OR PAST is applied, then those people will also come who satisfy the condition in their past jobs but not currently, along with those who satisfy the condition in their current job but not in their previous.

    For executive-level job titles (manager level and above), include 3-4 similar titles if it is clear that the user intends to hire for that role, rather than simply researching or looking it up, otherwise don't get any similar titles for executive titles (however always include abbreviations and full forms). For non-executive job titles (below manager level), we always include similar titles, based on the context of the prompt.
    
    For location requirements, if the specified location is a city, entire metropolitan area, entire state, entire country, or an entire continent, retain the location as stated. Our system is designed to handle cities, states, metropolitan areas, countries, and continents. However, if the requirement includes a region (e.g., "Eastern Europe"; part of a continent not the entire Europe) or mentions proximity (e.g., "near a location"), expand the location to include ALL relevant areas that meet the user's query and classify each location separately. For example, if the user specifies "London area," it will be interpreted as London city and london metro area only. However, if they say "near London," nearby cities will also be included. However, if an entire metropolitan area is referenced or a city is referenced do not get nearby cities or metros. Similarly, terms like "Middle East" or "MENA" would also be expanded, as our system handles only cities, states, metropolitan areas, countries, and continents. - For example: ***"30-mile radius of Miami FL"*** or ***"Nearby Miami FL"***: [ "Miami, Florida", "Miami Beach, Florida", "Coral Gables, Florida", "Doral, Florida", "Hialeah, Florida", "Homestead, Florida", "Key Biscayne, Florida", "North Miami, Florida", "North Miami Beach, Florida", "Aventura, Florida", "Sunny Isles Beach, Florida", "Sweetwater, Florida", "South Miami, Florida", "Miami Lakes, Florida", "Medley, Florida", "Opa-locka, Florida", "West Miami, Florida" ] will be returned (with state names included) as the user EXPLICITLY requires nearby regions.

    Output format: (return output enclosed in <Output> </Output> tags)
    <Output>
    { # Employee count controls the company size where you are looking for this title
    "job_title" : {
    'current' : [{"title_name" : "", "min_staff" : 0, "max_staff" : 50000000}], # The entities that must be in current, and not past experiences (must be logical job titles). (staff count controls the company size the user is looking for this title; 0 and 50000000 are default values) 
    'past' : [{"title_name" : "", "min_staff" : 0, "max_staff" : 50000000}], # The entities that must be in past, and not current experiences
    'both; : [{"title_name" : "", "min_staff" : 0, "max_staff" : 50000000}], # # The entities that can be in current, past both
    'event' : '' # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
    },
    "management_level" : {
    'current' : [], # The entities that must be in current, and not past experiences
    'past' : [], # The entities that must be in past, and not current experiences
    'both; : [], # # The entities that can be in current, past both
    'event' : '' # Can only be "CURRENT", "PAST", "CURRENT OR PAST", "CURRENT AND PAST"
    }, location will be exactly like management_level.
    .. # rest of the filters, if any
    }
    </Output>
</special_instructions>


"CURRENT OR PAST" and "CURRENT AND PAST" will only be applied when all the relevant entities are not just in current or just in past.  If all entities are in current then event has to be current and if all entities are in past, event has to be past. First give your reasoning for each entity and then the output. Assess job titles, locations, levels and industries all separately. When searching for candidates, treat required roles as current, on-going roles unless both current and past roles are explicitly defined separately.

If no temporal aspect is specified for an entity, then just assume "current" for the entity.

If skills are mentioned, then expand the skills. Include all the similar skills and abbreviations or full forms that we can search for.

Keep the rest of filters' lists (other than job_title, management_level and location) as they were; all filters should be included within the json object inside the output tags.
Our whole system is explained above. The filters should be extracted intelligently, ensuring highest recall and precision; for example if the prompt is "Find executives who are CEOs with experience in digital marketing," the mention of "digital marketing" experience specifies a skill or background but does not imply additional executive roles or overlapping titles. Therefore, the focus remains strictly on identifying individuals holding the title of "CEO" who also have experience in digital marketing. 
If you understand this system, respond with "YES."
"""

In [5]:
import anthropic

In [23]:
import os
from dotenv import load_dotenv
_ = load_dotenv()
client = anthropic.AsyncAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)


In [25]:
async def claude(query, retries=3):

    messages = [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": EXTRACT_USER_PROMPT}],
                },
                {"role": "assistant", "content": [{"type": "text", "text": "YES."}]},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"""Great! Here is the user query:\n\n\'''{query}'''\n\nFor each filter, first give your reasoning and then your output (ALWAYS a json object enclosed within <Output></Output> tags).\nThink logically about what the user requires, without decreasing the recall or precision, and apply filters and their current, past, and, or according to what would bring the best results to the user.""",
                        }
                    ],
                },
            ]
    message = await client.messages.create(
                model="claude-3-5-sonnet-latest",
                max_tokens=2048,
                messages=messages,
                temperature=0.1,
            )
    response = message.content[0].text
    # for i in range(retries):
    #     try:
    #         message = await client.messages.create(
    #             model="claude-3-5-sonnet-latest",
    #             max_tokens=2048,
    #             messages=messages,
    #             temperature=0.1,
    #         )
    #         response = message.content[0].text

    #         return response
    #     except Exception as e:
    #         pass
    # return {}
    return response


In [26]:
result = await claude(query="Looking for presidents who have also worked as a General Manager or SVP who have led a P&L in a relevant/ adjacent industrial tech company for at least five years of $500M+.")

In [36]:
res = eval(result[result.find('{'):result.rfind('}') + 1])

In [41]:
print(result)

Let me analyze each aspect of this query:

Job Titles Analysis:
- "Presidents" - Current requirement as it's the primary role being sought
- "General Manager" and "SVP" - Past experience requirement as indicated by "have also worked as"
- The roles need to be in sequence (past GM/SVP → current President) so this indicates "CURRENT AND PAST"
- For company size: "$500M+" indicates minimum revenue/size requirement
- Similar titles for President would include "Chief Executive", "CEO" as these are often interchangeable at this level

Management Levels Analysis:
- Not needed as specific titles are more precise than management levels
- "President" doesn't cover the entire "President" management level domain as we need those with specific experience
- "SVP" is part of "Executive VP or Sr. VP" but we need specific past experience, so better as title

Skills Analysis:
- "P&L" indicates P&L Management/P&L Leadership
- "Industrial tech" suggests Industrial Technology expertise

Total Working Years

In [37]:
res

{'job_title': {'current': [{'title_name': 'President',
    'min_staff': 500,
    'max_staff': 50000000},
   {'title_name': 'Chief Executive', 'min_staff': 500, 'max_staff': 50000000},
   {'title_name': 'CEO', 'min_staff': 500, 'max_staff': 50000000}],
  'past': [{'title_name': 'General Manager',
    'min_staff': 500,
    'max_staff': 50000000},
   {'title_name': 'Senior Vice President',
    'min_staff': 500,
    'max_staff': 50000000},
   {'title_name': 'SVP', 'min_staff': 500, 'max_staff': 50000000}],
  'both': [],
  'event': 'CURRENT AND PAST'},
 'skill': ['P&L Management',
  'P&L Leadership',
  'Industrial Technology',
  'Industrial Tech',
  'Business Unit Leadership'],
 'total_working_years': [5, 60]}

In [39]:
import anthropic
import os
import asyncio
from typing import List

client = anthropic.AsyncAnthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY"),
)


async def claude(query: str) -> str:
    
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": EXTRACT_USER_PROMPT}],
        },
        {"role": "assistant", "content": [{"type": "text", "text": "YES."}]},
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Great! Here is the user query:\n\n\'''{query}'''\n\nFor each filter, first give your reasoning and then your output (ALWAYS a json object enclosed within <Output></Output> tags).\nThink logically about what the user requires, without decreasing the recall or precision, and apply filters and their current, past, and, or according to what would bring the best results to the user.""",
                }
            ],
        },
    ]
    
    message = await client.messages.create(
        model="claude-3-5-sonnet-latest",
        max_tokens=2048,
        messages=messages,
        temperature=0.1,
    )
    return message.content[0].text

async def process_batch(queries: List[str], batch_size: int = 10) -> List[str]:
    """
    Process queries in batches asynchronously.
    
    Args:
        queries: List of query strings to process
        batch_size: Size of each batch (default: 10)
        
    Returns:
        List of Claude's responses
    """
    results = []
    
    # Process queries in batches
    for i in range(0, len(queries), batch_size):
        batch = queries[i:i + batch_size]
        
        # Create tasks for concurrent processing
        tasks = [claude(query) for query in batch]
        
        # Wait for all tasks in the batch to complete
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)
        
        # Optional: Add a small delay between batches to avoid rate limiting
        if i + batch_size < len(queries):
            await asyncio.sleep(1)
    
    return results


In [ ]:
with open('data')
queries = 

In [ ]:


# Process all queries
results = await process_batch(queries)

# Print results
for query, result in zip(queries, results):
    print(f"Query: {query}")
    print(f"Result: {result}")
    print("-" * 50)